In [1]:
%run ./CoreFunctions.ipynb

In [2]:
playsound(444,444)

No sound available because Linux


In [3]:

    
aLowestValue = -2.0
aHighestValue = 4.0
bLowestValue = -3.0
bHighestValue = 3.0

In [4]:
# Q values come below, after initialize cities array is defined

In [5]:
partition_size = 17  #377

In [6]:
SaveResults = False

In [7]:
#French population
Fr_Pop = pd.read_csv("summary_per_Aire_Attr_py_v55.csv")
FrPopulation = np.array(Fr_Pop["PAA"])
FrPopulation = np.sort(FrPopulation)

N_iterations = 39 
N_cities = len(FrPopulation)
#N_cities = 4 #number of city sizes

In [8]:
# kI  does not! affect the ratio I_p/I_u!
kI = 0.00001

In [9]:
# initialize cities array. The distribution is given manually
if False:
    def initialize_cities_array():
        
        # zeroth column is city size y, first is beta(y), second is pi_p(y), third  pi_U(y)
        zeta      = np.zeros((N_cities, 4))
        zeta[0,0] = 1000
        zeta[1,0] = 25369
        zeta[2,0] = 110521
        zeta[3,0] = 787445
        
        zeta[0,1] = 0.09
        zeta[1,1] = 0.05
        zeta[2,1] = 0.85
        zeta[3,1] = 0.01
        
        zeta[:,2] = 0.99 #placeholder
        zeta[:,3] = 0.99  #placeholder
    
    
        return zeta

In [10]:
# for Poland all city sizes are unique

print(len(list(FrPopulation)) > len(set(list(FrPopulation))))

if True:   
    def initialize_cities_array():
        
        # zeroth column is city size y, first is beta(y), second is pi_p(y), third  pi_U(y)
        zeta      = np.zeros((N_cities, 4))
        zeta[:,0] = FrPopulation
     
        
        zeta[:,1] = 1/N_cities
        
        zeta[:,2] = 0.99 #placeholder
        zeta[:,3] = 0.99  #placeholder
    
    
        return zeta

True


In [11]:
Q_values = np.array([ 0.03, 0.11, 0.16, 0.21, 0.255, 0.32, 0.41, 0.57, 0.76]) * maxQ_value()
#Q_values = np.array([0.3,0.88]) * maxQ_value()  82358. 85637. 87051. 88134. 89370
Q_values_from_Aurelien = np.array([82358,  87051,  89370])
Q_values = np.concatenate([Q_values, Q_values_from_Aurelien])

In [12]:
Q_values

array([ 2698.92696,  9896.06552, 14394.27713, 18892.48873, 22940.87917,
       28788.55425, 36885.33513, 51279.61226, 68372.81635, 82358.     ,
       87051.     , 89370.     ])

In [13]:
Q_max_possible_value = maxQ_value()

In [14]:
ick = Compute_Is(-0.432, 1.2315, 6894)
math.log(ick[0]) - math.log(ick[1])

3.287199874050975

In [15]:
Q_max_possible_value

89964.23203592816

In [16]:
if np.max(Q_values) > maxQ_value() - 0.1:
    raise ValueError("Q_value is too high!")

In [17]:
#determine boundaries using the bisection algorithm

aaaaaaas = np.linspace(start=aLowestValue , stop=aHighestValue   , num=partition_size) 
bbbbbbbs = np.linspace(start=bLowestValue , stop=bHighestValue, num=partition_size) 



aas = np.array([aaaaaaas for i in range(np.shape(Q_values)[0])])
bbs = np.array([bbbbbbbs for i in range(np.shape(Q_values)[0])])

bbs_of_aas = bLowestValue* np.ones(np.shape(aas)) 
aas_of_bbs = -2* np.ones(np.shape(bbs))




In [18]:
aANDbLimit = 26

In [19]:
def Find_approx_root_given_a(a, Tolerance, Q_value, Verbose = False):
    
    # given a, finds an approx value of b such that Q_U =Q_P = Q_value and Q_P =Q_U
    
    b = np.random.uniform(low = -0.22*aANDbLimit, high = 0.22*aANDbLimit)
    Continue_while_loop = True
    
    Iteration_number = 0
    
    high = aANDbLimit
    low = -aANDbLimit
    
    while Iteration_number < 60:
        
        if b > 0.995* aANDbLimit:
            b = 1001
            break
            
        if b < -0.995* aANDbLimit:
            b = -1001
            break
            
        Iteration_number = Iteration_number + 1
        ick = Compute_Is(a, b, Q_value)
        LogRatio = math.log(ick[0]) - math.log(ick[1])
        if Verbose == True:
            print("b = " + str(b)+ ", LogRatio = " + str(LogRatio))
        if abs(LogRatio) < Tolerance: 
            Continue_while_loop = False
            break 
            
        elif LogRatio >= Tolerance:
            low = b
            b = 0.5*(b + high)
        else: 
            high = b
            b = 0.5*(b + low)
            

            
        if  Iteration_number == 42:
            print("Iteration number searching for b is too high. Check whats going on.  ")
            print("Possibly a and b Limits need to be increased  ")
            print("a , Tolerance, Q_value = " +   str([a, Tolerance, Q_value]))    
        
    return b
        
def Find_approx_root_given_b(b, Tolerance, Q_value, Verbose = False):
    
    # see Find_approx_root_given_a's description
    a = np.random.uniform(low= -0.22*aANDbLimit, high= 0.22*aANDbLimit)
    Continue_while_loop = True
    
    Iteration_number = 0
    
    high = aANDbLimit
    low = -aANDbLimit
    
    while Iteration_number < 60:
        
        if a > 0.995* aANDbLimit:
            a = 1001
            break
        
        if a < -0.995* aANDbLimit:
            a = -1001
            break
            
        Iteration_number = Iteration_number + 1
        ick = Compute_Is(a, b, Q_value)
        LogRatio = math.log(ick[0]) - math.log(ick[1])
        if Verbose == True:
            print("a = " + str(a)+ ", LogRatio = " + str(LogRatio))
        if abs(LogRatio) < Tolerance: 
            Continue_while_loop = False
            break 
            

            
        elif LogRatio >= Tolerance:
            low = a
            a = 0.5*(a + high)
        else: 
            high = a
            a = 0.5*(a + low)
        
        if  Iteration_number == 42:
                print("Iteration number searching for a is too high. Check whats going on.  ")
                print("Possibly a and b Limits need to be increased  ")
                print("b , Tolerance, Q_value = " +   str([b, Tolerance, Q_value]))
        
    return a
        

In [20]:
maxQ_value()

89964.23203592816

In [ ]:
# extremal value the root search function can take
# relies on the ration log (I_p/I_u) to be monotonically decreasing in a and b!

print("This loop relies on the ratio log (I_p/I_u) to be monotonically decreasing in a and b!")



Tolerance = 0.00065 # a lower tolerance doesnt make much sense without decreasing tolerance of the core functions

for k in range(np.shape(Q_values)[0]):
    print("k = " + str(k))
    Q_value = Q_values[k]
    startTime = time.time()
    print(" ")
        
    
    for i in range(np.shape(aas)[1]):
        a = aas[k][i]
        bbs_of_aas[k][i]  = Find_approx_root_given_a(a, Tolerance, Q_value)
        
        if  i%73 == 5:
            print("ETA for LogRation computation given a in this subloop = " +
                      str( (np.shape(aas)[1] - i - 1) *(time.time() - startTime)/(i+1) ) + " seconds")
            print("Average number of seconds per iteration in this subloop = "
                  + str((time.time() - startTime)/(i+1)) + " seconds")
            print(" ")
            
    print("Total time for this subloop = " + str(time.time() - startTime) + " seconds")
    print("Average number of seconds per iteration in this subloop = "
          + str((time.time() - startTime)/(i+1)) + " seconds")
    print(" ") 
    
    startTime = time.time()
    for i in range(np.shape(bbs)[1]):
        
        b = bbs[k][i]
        aas_of_bbs[k][i] = Find_approx_root_given_b(b, Tolerance, Q_value)
        
        if  i%73 == 5:
            print("ETA for LogRation computation given b in this subloop = " +
                      str( (np.shape(aas)[1] - i - 1) *(time.time() - startTime)/(i+1) ) + " seconds")
            print("Average number of seconds per iteration in this subloop = "
                  + str((time.time() - startTime)/(i+1)) + " seconds")
            print(" ")
            
    print("Total time for this subloop = " + str(time.time() - startTime) + " seconds")
    print("Average number of seconds per iteration in this subloop = "
          + str((time.time() - startTime)/(i+1)) + " seconds")
    print(" ")


This loop relies on the ratio log (I_p/I_u) to be monotonically decreasing in a and b!
k = 0
 
ETA for LogRation computation given a in this subloop = 120.45375871658325 seconds
Average number of seconds per iteration in this subloop = 10.950353980064392 seconds
 
Total time for this subloop = 143.9709038734436 seconds
Average number of seconds per iteration in this subloop = 8.468881803400377 seconds
 
ETA for LogRation computation given b in this subloop = 108.535493572553 seconds
Average number of seconds per iteration in this subloop = 9.866876045862833 seconds
 
Total time for this subloop = 136.84922170639038 seconds
Average number of seconds per iteration in this subloop = 8.04995846748352 seconds
 
k = 1
 


In [ ]:
playsound(364,444)

In [ ]:
import matplotlib.patches as mpatches

In [ ]:


palette = sns.color_palette(None, np.shape(Q_values)[0])

patches = []

for k in range(0,np.shape(Q_values)[0]):

    patches.append(mpatches.Patch(color = palette[k], label ='{0:.1f}'.format(Q_values[k])))
        
    plt.plot(aas[k][(bbs_of_aas[k] < 0.99* aANDbLimit)  & (bbs_of_aas[k] > -0.99*aANDbLimit) ],
             bbs_of_aas[k][(bbs_of_aas[k] < 0.99* aANDbLimit)  & (bbs_of_aas[k] > -0.99*aANDbLimit) ], 
             color=palette[k], linewidth=1.0)
    plt.plot( aas_of_bbs[k][(aas_of_bbs[k]< 0.99* aANDbLimit) & (aas_of_bbs[k]> -0.99* aANDbLimit)] , 
             bbs[k][(aas_of_bbs[k]< 0.99* aANDbLimit) & (aas_of_bbs[k]> -0.99* aANDbLimit)],
             color=palette[k], linewidth=1.0)
    
    
plt.ylabel('b')
plt.xlabel('a')
plt.legend(handles=patches,  title="Q values", bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)


plt.title("Boundaries for different values of Q")
if SaveResults == True:
    plt.savefig('Boundaries for various Q' + str(int(time.time()//1)) + '.png', bbox_inches='tight')

plt.show()    

In [ ]:
    if SaveResults == True:
        fileObj = open('Poland_Boundaries_different_Q_'  + str(int(time.time()//1)) + '.pkl', 'wb')
        pickle.dump([aas,bbs_of_aas, bbs, aas_of_bbs],fileObj)
        fileObj.close()

In [ ]:
time.sleep(2)

In [ ]:
XandYlims = 8

In [ ]:


palette = sns.color_palette(None, np.shape(Q_values)[0])

patches = []

for k in range(0,np.shape(Q_values)[0]):
    if k == 2:
        continue
    patches.append(mpatches.Patch(color = palette[k], label ='{0:.1f}'.format(Q_values[k])))
        
    plt.plot(aas[k][(bbs_of_aas[k] < XandYlims)  & (bbs_of_aas[k] > -XandYlims) ],
             bbs_of_aas[k][(bbs_of_aas[k] < XandYlims)  & (bbs_of_aas[k] > -XandYlims) ], 
             color=palette[k], linewidth=1.0)
    plt.plot( aas_of_bbs[k][(aas_of_bbs[k]< XandYlims) & (aas_of_bbs[k]> -XandYlims)] , 
             bbs[k][(aas_of_bbs[k]< XandYlims) & (aas_of_bbs[k]> -XandYlims)],
             color=palette[k], linewidth=1.0)
    
    
plt.ylabel('b')
plt.xlabel('a')
plt.legend(handles=patches,  title="Q values", bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)


plt.title("Boundaries for different values of Q")
if SaveResults == True:
    plt.savefig('Boundaries for various Q' + str(int(time.time()//1)) + '.png', bbox_inches='tight')

plt.show()    

In [ ]:
time.sleep(2)
XandYlims = 2

palette = sns.color_palette(None, np.shape(Q_values)[0])

patches = []

for k in range(0,np.shape(Q_values)[0]):

    patches.append(mpatches.Patch(color = palette[k], label ='{0:.1f}'.format(Q_values[k])))
        
    plt.plot(aas[k][(bbs_of_aas[k] < XandYlims)  & (bbs_of_aas[k] > -XandYlims) ],
             bbs_of_aas[k][(bbs_of_aas[k] < XandYlims)  & (bbs_of_aas[k] > -XandYlims) ], 
             color=palette[k], linewidth=1.0)
    plt.plot( aas_of_bbs[k][(aas_of_bbs[k]< XandYlims) & (aas_of_bbs[k]> -XandYlims)] , 
             bbs[k][(aas_of_bbs[k]< XandYlims) & (aas_of_bbs[k]> -XandYlims)],
             color=palette[k], linewidth=1.0)
    
    
plt.ylabel('b')
plt.xlabel('a')
plt.legend(handles=patches,  title="Q values", bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)


plt.title("Boundaries for different values of Q")
if SaveResults == True:
    plt.savefig('Boundaries for various Q' + str(int(time.time()//1)) + '.png', bbox_inches='tight')

plt.show()    

In [ ]:
time.sleep(2)


palette = sns.color_palette(None, np.shape(Q_values)[0])

patches = []

for k in range(0,np.shape(Q_values)[0]):

    patches.append(mpatches.Patch(color = palette[k], label ='{0:.1f}'.format(Q_values[k])))
        
    plt.plot(aas[k], bbs_of_aas[k], 
             color=palette[k], linewidth=1.0)
    plt.plot( aas_of_bbs[k], bbs[k],
             color=palette[k], linewidth=1.0)

plt.xlim(0.7,1.3)
plt.ylim(-0.3,0.3)
plt.ylabel('b')
plt.xlabel('a')
plt.legend(handles=patches,  title="Q values", bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0)


plt.title("Boundaries for different values of Q")
if SaveResults == True:
    plt.savefig('Boundaries for various Q' + str(int(time.time()//1)) + '.png', bbox_inches='tight')

plt.show()    

In [ ]:
maxQ_value()

In [ ]:
# recompute for k = 2 with a lower tolerance